# 🧠 ML Demo: Dagelijkse Productiviteit Voorspellen

In deze demo bouwen we een machine learning-model dat de productiviteit van een persoon voorspelt op basis van dagelijkse gewoonten zoals slaap, schermtijd, pauzes, etc.

## 📊 Stap 1: Data genereren en bekijken

We gebruiken gesimuleerde data met de volgende variabelen:
- `slaap_uren`: aantal uur geslapen
- `schermtijd_uren`: tijd gespendeerd aan schermen
- `pauzes_genomen`: aantal pauzes op een dag
- `aantal_meetings`: vergaderingen per dag
- `stemmingscore`: hoe goed voelt iemand zich (1-10)
- `productiviteit`: doelvariabele, score van 0 (niet productief) tot 10 (heel productief)

In [ ]:
import pandas as pd
import numpy as np

np.random.seed(42)
data = {
    'slaap_uren': np.random.normal(7, 1, 100),
    'schermtijd_uren': np.random.normal(6, 1.5, 100),
    'pauzes_genomen': np.random.randint(0, 5, 100),
    'aantal_meetings': np.random.randint(0, 6, 100),
    'stemmingscore': np.random.uniform(1, 10, 100),
}

df = pd.DataFrame(data)
df['productiviteit'] = (
    0.3 * df['slaap_uren'] -
    0.2 * df['schermtijd_uren'] +
    0.1 * df['pauzes_genomen'] -
    0.05 * df['aantal_meetings'] +
    0.4 * df['stemmingscore'] +
    np.random.normal(0, 0.5, 100)
).clip(0, 10).round(1)

df.head()

## 🔍 Wat doet "np.random.normal()" precies?
Dit gebruikt NumPy om 100 willekeurige getallen te genereren die ongeveer lijken op echte meetwaarden – in dit geval slaapuren.

💡 np.random.normal(mean, std, size)
mean = 7: het gemiddelde van de gegenereerde waarden (dus ~7 uur slaap)

std = 1: de standaarddeviatie, ofwel: hoeveel variatie er is (tussen ±1 uur)

size = 100: het aantal waarden dat we genereren

## ✏️ Hoe pas je dit aan aan jouw eigen data?
**Stel**:

Je wil data maken met een gemiddelde van 6,5 uur slaap

Je wil iets minder variatie, zeg 0.8 uur

Je wil dit voor 365 dagen doen (één jaar)

Dan pas je de code zo aan:

slaap_uren = np.random.normal(6.5, 0.8, 365)

## 🧹 Stap 2: Data splitsen in train en test

We verdelen onze data zodat we kunnen trainen op één deel en testen op een ander deel om overfitting te vermijden.


In [ ]:
from sklearn.model_selection import train_test_split

X = df.drop(columns='productiviteit')
y = df['productiviteit']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## 🔍 Wat doen we hier?
🧠 Machine Learning-modellen leren van voorbeelden
Je wilt een model dat leert van data om voorspellingen te doen.

Maar: je kunt de kwaliteit van dat model alleen eerlijk beoordelen als je test op data die het nog niet heeft gezien tijdens het trainen.

🎯 Daarom splitsen we de data in:
Trainingsset (bijv. 80%) → om het model te trainen

Testset (bijv. 20%) → om het model te evalueren

## 🧠 Stap 3: Model trainen

We gebruiken een eenvoudige regressie (Random Forest) om een numerieke score te voorspellen.

In [ ]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

## 🌳 Wat is een Random Forest?
Een Random Forest is een krachtig en flexibel machine learning-model dat voorspellingen doet door veel beslisbomen te combineren.

## 🔍 In het kort:
Een beslisboom is een model dat beslissingen neemt door vragen te stellen zoals:

*“Heeft deze persoon meer dan 6 uur geslapen?”*

*“Is de schermtijd groter dan 8 uur?”*


Eén boom is vaak niet zo precies (kan overfitten of te simplistisch zijn).

Een Random Forest maakt veel verschillende bomen (bijvoorbeeld 100), elk op een iets andere subset van de data.

## 🌲 Het woord 'random' slaat op:
Elke boom krijgt een willekeurige selectie van datapunten uit de trainingsset.

Bij het splitsen in de boom wordt op een willekeurige subset van features gezocht naar de beste splitsing.

De voorspelling van het model is dan het gemiddelde van al die bomen (voor regressie).

## 🎯 Waarom gebruiken we Random Forest?
✅ Voordelen:

Werkt goed met weinig tuning

Kan niet-lineaire verbanden leren

Robuust tegen overfitting (beter dan één enkele boom)

Kan feature importance tonen (welke variabelen het belangrijkst zijn)

❌ Nadelen:

Niet altijd makkelijk uitlegbaar

Kan traag zijn bij hele grote datasets

## ✅ Stap 4: Resultaten bekijken

We meten hoe goed het model presteert met R²-score en bekijken enkele voorspellingen.

In [ ]:
from sklearn.metrics import r2_score

y_pred = model.predict(X_test)
score = r2_score(y_test, y_pred)
print(f"R²-score: {score:.2f}")

# Vergelijk echte vs voorspelde waarden
output = pd.DataFrame({'Echt': y_test.values, 'Voorspeld': y_pred.round(1)})
output.head()

## 🔍 Wat doen we hier precies?

## 🔢 1. model.predict(X_test)
Hiermee gebruik je het getrainde model om voorspellingen te doen op de testdata – dus op data die het model nog niet heeft gezien.

Voor elke rij (dag) voorspelt het model een productiviteitsscore tussen 0 en 10.

## 🎯 2. r2_score(y_test, y_pred)
Dit is de R-kwadraat score (coëfficiënt van determinatie) – een maat voor hoe goed het model de variatie in de data verklaart.

🔍 Interpretatie van R²:
1.0 = perfect model (voorspelt alles exact goed)

0.0 = model doet het niet beter dan een simpel gemiddelde

Negatief = model doet het slechter dan een domme gok


## 📊 3. pd.DataFrame({'Echt': ..., 'Voorspeld': ...})
Dit toont een vergelijking tussen:

Echt: wat er daadwerkelijk gemeten of vastgelegd is

Voorspeld: wat het model denkt dat het zou moeten zijn

💡 Dit helpt om te zien:

Zit het model er meestal dichtbij?

Zijn er systematische overschattingen of onderschattingen?


## 🧠 Waarom is dit belangrijk?
Je leert hoe goed je model generaliseert naar nieuwe situaties

Je voorkomt dat je denkt dat het model slim is, terwijl het alleen maar trainingsdata heeft gememoriseerd

Je kunt feedback geven aan het model (bijv. door extra features toe te voegen als het nog niet goed genoeg is)



## 📈 Scatterplot: Voorspeld vs. Werkelijk

Laat zien hoe goed de voorspellingen overeenkomen met de echte waarden.
Een perfecte voorspelling ligt precies op de diagonale rode lijn.

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(6, 6))
plt.scatter(y_test, y_pred, alpha=0.7)
plt.xlabel('Werkelijke productiviteit')
plt.ylabel('Voorspelde productiviteit')
plt.title('Voorspeld vs Werkelijk')
plt.plot([0, 10], [0, 10], '--r')  # Diagonale lijn als referentie (perfecte score)
plt.grid(True)
plt.show()

## 🧪 Bonus: Voorspel jouw productiviteit

Je kunt nu zelf een dag invullen en laten voorspellen hoe productief je zal zijn.

In [ ]:
voorbeeld = pd.DataFrame({
    'slaap_uren': [7.5],
    'schermtijd_uren': [5],
    'pauzes_genomen': [2],
    'aantal_meetings': [3],
    'stemmingscore': [6.5]
})

voorspelling = model.predict(voorbeeld)
print(f"Voorspelde productiviteitsscore: {voorspelling[0]:.1f} / 10")